In [3]:
!pip install numpy
!pip install tensorflow
!pip install librosa
!pip install soundfile
!pip install tdqm


In [8]:
import zipfile
import os

def unzip_file(zip_file_name):
    with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
        zip_ref.extractall(os.path.splitext(zip_file_name)[0])

# Example usage:
zip_file_name = 'triad_spectrograms.zip'

unzip_file(zip_file_name)

In [4]:
!pip install pillow
!pip install matplotlib

In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from generateTriadSpectrograms import generate_spectrograms_for_directory

2023-11-28 14:53:11.430644: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-28 14:53:12.175385: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 14:53:13.258222: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 14:53:13.258273: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 14:53:13.478392: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:


# Assuming the label_simplified_spectrograms function is defined in the file
from generateTriadSpectrograms import label_simplified_spectrograms

import os
import librosa
import soundfile as sf

def trim_audio_files_in_directory(input_directory, output_directory=None, target_sr=22050):
    if not os.path.isdir(input_directory):
        print(f"The directory {input_directory} does not exist.")
        return

    if output_directory is None:
        output_directory = input_directory
    else:
        os.makedirs(output_directory, exist_ok=True)

    for file in os.listdir(input_directory):
        if file.endswith('.wav'):
            input_file_path = os.path.join(input_directory, file)
            output_file_path = os.path.join(output_directory, file)

            # Load the audio file
            y, sr = librosa.load(input_file_path, sr=target_sr)

            # Trim the audio file
            y_trimmed, _ = librosa.effects.trim(y)

            # Save the trimmed audio using soundfile
            sf.write(output_file_path, y_trimmed, sr)


# Example usage
input_dir = 'triad_database'
output_dir = 'trimmed_triad'  # Set to None to overwrite original files
"""
trim_audio_files_in_directory(input_dir, output_dir)
"""
"""
# 1. Generate Spectrograms
input_directory = 'trimmed_triad'
output_directory = 'triad_spectrograms'
generate_spectrograms_for_directory(input_directory, output_directory)
"""
import os
import re
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
from tqdm import tqdm

output_directory = 'triad_spectrograms'

# 2. Label Spectrograms
labels = label_simplified_spectrograms(output_directory)



Labeling Spectrograms: 100%|██████████| 43200/43200 [00:00<00:00, 1483846.11it/s]


In [3]:
from sklearn.preprocessing import LabelEncoder
# 3. Load and Preprocess Data
def load_and_preprocess_data(directory, labels):
    images = []
    image_labels = []

    # Use tqdm for a progress bar
    for filename, label in tqdm(labels.items(), desc="Processing Images", unit="image"):
        filepath = os.path.join(directory, filename)
        image = Image.open(filepath)
        image = np.array(image) / 255.0  # Normalize the images
        images.append(image)

        # Convert labels to the necessary type (int)
        image_labels.append(label)

    # Encode unique labels into integers
    label_encoder = LabelEncoder()
    image_labels_encoded = label_encoder.fit_transform(image_labels)

    return np.array(images), np.array(image_labels_encoded), label_encoder

images, image_labels, label_encoder = load_and_preprocess_data(output_directory, labels)

Processing Images:  41%|████      | 17532/43200 [09:13<62:29:30,  8.76s/image]IOStream.flush timed out


: 

: 

In [ ]:
print("Splitting")
X_train, X_val, y_train, y_val = train_test_split(images, image_labels, test_size=0.2)
print("split")
# 4. Build and Train CNN Model
def build_cnn_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

print("Starting to build the model")
num_classes = len(set(image_labels))
model = build_cnn_model(X_train.shape[1:], num_classes)
print("Model Built")
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val),verbose=1)
print("Model trained")

# 5. Save the Trained Model
model.save('triadModel.h5')

# Save the label encoder for future use
joblib.dump(label_encoder, 'label_encoder.joblib')

2023-11-28 01:34:33.432115: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
